In [ ]:
import pandas
import numpy
import openai
import requests
import json
import openai

# ="input your own openai api key. Github do not allow to contain secret info"

In [2]:
def call_GPT(query, answer_format, temperature=0.5, model="gpt-4-turbo", DNA=True, max_token=150, specialist=None):
    """
    Ask GPT to answer the input question in a required format.

    Parameters:
    - query: The question or prompt you want GPT to answer.
    - answer_format: The format in which you'd like GPT's response to be structured.
    - max_token: The maximum number of tokens in GPT's response to control response length.
    - model : The ChatGPT model you want to use

    Returns:
    - response: GPT's response to the query, formatted as specified by `answer_format` and within the `max_token` limit.
    """
    
    if DNA:
        messages = [
        {
            "role": "user",
            "content": f'{query}, please answer the question in the following format: "{answer_format}", do not give anything except json object. Answer within maxium token of {max_token}.',
        }
        ]
    else:
        messages = [
        {
            "role": "user",
            "content": f'{query}, please answer the question in the following format: "{answer_format}", do not give anything except json object. Answer within maxium token of {max_token}',
        },
        {
            "role": "system", 
            "content": f'You are a {specialist}.'
            }
        ]
        
    response = openai.chat.completions.create(
    messages = messages,
    model = model,
    max_tokens = max_token
    )
    content = response.choices[0].message.content
    
    return content

In [3]:
def DNA_node(query, max_node_number = 10, summary = False, specialist_group = None, concentrated_response = None, max_token = 200):
    '''
    activate DNA node to specify roles for available nodes

    Parameters:
    - query: Ask GPT to decide the number of specialists it needs to solve the problem and the role of each specialist
    - max_node_number: The max number of nodes you want to activate

    Returns:
    - specialists: A Json object containing the the information of specialist group
    '''

    if not summary:
        prompt = '''You are assigned a particular task as follows: "{query}", and you can use up to {max_node_number} specialists to solve this task. 
                    Please decide the number of specialists you want to use and specify their rolls (e.g mathmatician, physician, psychologist etc.).Use more specilists in one particular field if you think it is needed. 
                    Please also balance the cost and the efficiency when using the specialists.
                    '''.format(query=query, max_node_number=max_node_number)
        
        answer_format = {
            "number of specialists": 6,
            "role of the specialists": ["mathmatician", "physician"],
            "number of each specialists": [2,4],
            "reason": "explaine why you want to use such many specialists and seperate them in this way",
            "question": "repeat what is the query"
                        }
        
        specialists = call_GPT(query=prompt, answer_format=answer_format, max_token=max_token, model="gpt-4-turbo")
        
        return specialists
    
    else:
        prompt = '''
                You previously designated a specialist group as follows: "{specialist_group}" to solve this question: "{query}". Here are the answers from the specialist group: "{concentrated_response}". 
                Combine all the answers into one answer. Try to balance details and length. Give your answer within {max_token} tokens. 
                '''.format(specialist_group = specialist_group, query = query, concentrated_response = concentrated_response, max_token = max_token)
        
        answer_format = {
            "combined answer": "write your combined answer here",
            "satisfactory": "Y/N, answer Y if you are satisfied with this answer. Answer N otherwise",
            "debate/rearrange": "D/R/NA, answer D if you want to maintain the current structure of specialist group and start a new round of discussion. Answer R if you want to rearrange the specialist group and start from the beginning. Answer R only when you think the current specialist group cannot handle this question properly. Answer NA if you think the current answer is good enough.",
        }

        response = call_GPT(query=prompt, answer_format=answer_format, max_token=max_token, model="gpt-4-turbo")

        return response

In [4]:
def specialists_divider(response):
    '''
    Divide the roles for specialists based on the decision of DNA node

    Parameters:
    - response: The response of DNA node
    - total_number: The total number of specialists needed by DNA node

    Returns:
    - specialist_group : a dictionary where keys are the role and contents are their correponding number
    - total number : The total number of specialists required
    '''
    response_json = json.loads(response)
    specialists = response_json['role of the specialists']
    numbers = response_json['number of each specialists']

    specialists_group = {}

    for i, specialist in enumerate(specialists):
        specialists_group[specialist] = numbers[i]

    total_number = response_json['number of specialists']

    return specialists_group, total_number


In [5]:
def stem_cell_node(query, answer_format, specialists_group, debate=False, specialist=None, model="gpt-4-turbo", max_token=150):
    """
    Activate stem_cell_node according to the decision made by DNA node to answer specific questions

    Parameters:
    - query: The question or prompt you want GPT to answer.
    - answer_format: The format in which you'd like GPT's response to be structured.
    - max_token: The maximum number of tokens in GPT's response to control response length.
    - model: The ChatGPT model you want to use
    - specialist: The specified role of this node(etc. Physician)

    Returns:
    - response: GPT's response to the query, formatted as specified by `answer_format` and within the `max_token` limit, the final response should be a python dictionary
    """
    if not debate:
        response = {}
        for specialists in specialists_group:
            response[specialists] = []
            for i in range(specialists_group[specialists]):
                ans = call_GPT(query=query, answer_format=answer_format, model=model, DNA=False, max_token=max_token, specialist=specialists)
                response[specialists].append(ans)

    if debate:
        response = {}
        for i in range(specialists_group[specialist]):
            ans = call_GPT(query=query, answer_format=answer_format, model=model, DNA=False, max_token=max_token, specialist=specialist)
            response[specialist] = []
            response[specialist].append(ans)


    return response


In [6]:
def ans_combiner(ans, answer_format, model = "gpt-4-turbo", max_token = 200):
    query = '''
            Combine the following answer into one answer: {ans}. Try to balance details and length. Give your answer within {max_token} tokens
            '''.format(ans = ans, max_token = max_token)
    
    response = call_GPT(query=query, answer_format=answer_format, model=model, DNA=False, max_token=max_token)

    return response
    

In [20]:
query = 'analyze the current GDP trend of China'
answer_format = {
    "confidence_rate": "0%-100% based on your confidence to your response, if you do not have confidence at all write 0% here. Only input integer here.",
    # adding confidence_rate will make GPT more cautious, and therefore give equalvocal response. I decided to mark it off to enhence the perfomance. 
    "response": "write your reponse here"}

In [9]:
specialists = DNA_node(query=query)
specialists

'{\n    "number of specialists": 6,\n    "role of the specialists": ["economist", "data analyst", "political scientist", "sociologist"],\n    "number of each specialists": [2, 2, 1, 1],\n    "reason": "Economists are crucial for understanding economic theories and GDP components. Data analysts will handle and interpret GDP data trends and projections efficiently. A political scientist will provide insights on government policies affecting GDP. A sociologist will analyze social factors influencing economic activities. This distribution ensures a comprehensive and multidisciplinary analysis balancing expertise and cost-efficiency.",\n    "question": "analyze the current GDP trend of China"\n}'

In [10]:
specialists_group, total_number = specialists_divider(specialists)


In [21]:
response = stem_cell_node(query=query, answer_format=answer_format, specialists_group=specialists_group)
response

{'economist': ['{\n  "confidence_rate": "75%",\n  "response": "As of 2023, China\'s GDP growth has shown signs of slowing compared to its pre-pandemic rates, primarily due to domestic challenges such as property sector debt and COVID-19 disruptions. The government is implementing regulatory measures and economic policies to stabilize growth."\n}',
  '{\n  "confidence_rate": "85%",\n  "response": "China\'s GDP growth has been slowing in recent times due to various internal and external challenges including regulatory crackdowns, real estate sector troubles, and global economic pressures. However, it remains robust compared to many other major economies."\n}'],
 'data analyst': ['{\n  "confidence_rate": "75%",\n  "response": "China\'s GDP trend has seen fluctuations but generally maintains a growth trajectory. Recent trends indicate a deceleration influenced by various factors including COVID-19 impacts, regulatory tightening in certain sectors, and global economic pressures. However, su

In [ ]:
concentrated_response = {}
new_response = {}
combined_response = {}
for specialist in response:
    if len(response[specialist]) == 1:
        concentrated_response[specialist] = response[specialist]
        print(f'Answer for {specialist} recorded!')
    else:
        new_response[specialist] = []
        combined_response[specialist] = []
        for i in range(len(response[specialist])):
            query = '''
                    Here are the responses from other {specialist}s in your group: "{response1}". Here is your original response: "{response2}"Do you think you can improve your response?
                '''.format(specialist = specialist, response1 = response[specialist][i], response2 = response[specialist][:i] + response[specialist][i+1:])
            answer_format = {
                "improvement": "Y/N, answer Y if you think your original response needs improvement. Answer N otherwise",
                "response": "update your response",
                "reason": "Where have you improved? What have you learnt from the other specialists?",
                "further_improvement": "Y/N, answer Y if you think your updated response needs further improvement. Answer N otherwise"
            }
            ans = stem_cell_node(query=query, answer_format=answer_format, specialists_group=specialists_group, debate=True, specialist=specialist, max_token=400)
            new_response[specialist].append(ans)
            for item in new_response[specialist]:
                combined_response[specialist].extend(item[specialist])
            
        total = 0
        N = 0
        for item in combined_response[specialist]:
            total +=1
            json_item = json.loads(item)
            if json_item['further_improvement'] == 'N':
                N +=1
        print(N/total)
        if N/total >= 0.5:
            print(f'{specialist} arrived in the final conclusion')
            ans_format = {
                # "confidence_rate": "0%-100% based on your confidence to your response, if you do not have confidence at all write 0% here. Only input integer here.",
                # adding confidence_rate will make GPT more cautious, and therefore give equalvocal response. I decided to mark it off to enhence the perfomance. 
                "response": "write your reponse here"}
            concentrated_response[specialist] = ans_combiner(combined_response[specialist], answer_format=ans_format)
            
            

            



1.0
economist arrived in the final conclusion
1.0
data analyst arrived in the final conclusion
Answer for political scientist recorded!
Answer for sociologist recorded!


In [17]:
query = 'analyze the current GDP trend of China'

In [18]:
final_response = DNA_node(query=query, summary=True, specialist_group=specialists_group, concentrated_response=concentrated_response)

In [19]:
final_response

'{\n    "combined answer": "As of 2023, China\'s GDP growth trend indicates stability and recovery post-COVID-19, though the pace has moderated compared to historical rates. Factors such as aging demographics, increased debt, stringent regulations in key sectors, trade tensions, and global economic shifts have slowed growth. Notably, domestic challenges like real estate volatility and regulatory changes impact tech and private sectors. In response, China is steering its economic policy towards boosting domestic consumption, technology advancement, and export competitiveness, transitioning from an investment-heavy to a consumption-driven model for sustainable growth.",\n    "satisfactory": "Y",\n    "debate/rearrange": "NA"\n}'

In [102]:
concentrated_response

{'economist': '```json\n{\n  "response": "As of 2023, China\'s GDP shows steady but slowing growth due to demographic shifts, rising debt, and trade tensions, mainly with the US. The COVID-19 pandemic\'s lingering effects also impact economic activities. In response, the government has boosted domestic consumption, fostered technological innovation, and promoted sustainable practices. These measures aim to stabilize and future-proof the economy amidst global uncertainties and internal challenges, maintaining a focus on long-term economic health over short-term gains."\n}\n```',
 'data analyst': '```json\n{\n    "response": "As of my last update prior to 2023, China\'s GDP growth has been influenced by multiple factors. Initially, there was a sharp decline due to the COVID-19 pandemic, followed by a rapid rebound facilitated by aggressive public health measures and extensive fiscal stimulus. However, the post-recovery period has displayed more moderate growth rates due to regulatory tig

'```json\n{\n  "response": "As of 2023, China\'s GDP shows steady but slowing growth due to demographic shifts, rising debt, and trade tensions, mainly with the US. The COVID-19 pandemic\'s lingering effects also impact economic activities. In response, the government has boosted domestic consumption, fostered technological innovation, and promoted sustainable practices. These measures aim to stabilize and future-proof the economy amidst global uncertainties and internal challenges, maintaining a focus on long-term economic health over short-term gains."\n}\n```'

AttributeError: 'str' object has no attribute 'read'

In [38]:
combined_response

{'economist': ['{\n    "improvement": "Y",\n    "response": "As of the latest updates up until 2023, China\'s GDP has shown signs of gradual growth, although at a more moderate pace compared to previous decades. The country is navigating numerous economic challenges including demographic shifts, rising debt levels, and trade tensions with major economies such as the U.S. Additionally, impacts from the COVID-19 pandemic and internal policy adjustments aimed at technology regulation and real estate market stabilization have also influenced economic trends. In response, the Chinese government has been focusing on policies that boost domestic consumption, invest in technological advancements, and promote sustainable economic practices, aiming to secure stability and sustain growth.",\n    "reason": "The improved response synthesizes elements from the other economist’s remarks, particularly emphasizing the variety of influences on China’s economic trajectory such as trade tensions, pandemic

In [39]:
concentrated_response

{'political analyst': ["{'response': 'As of the latest data before my last update in 2023, China's GDP growth showed signs of slowdown compared to its performance in previous decades. The slowdown was influenced by various factors such as demographic changes, with a growing aging population, an increase in debt levels, and trade tensions with other major economies, including the United States. Additionally, China's government has been focused on implementing regulatory reforms and controlling high-risk financial practices, which also affect short-term economic growth rates. Despite these challenges, the country continued to maintain a positive GDP growth rate, albeit at a lower rate than the high growth experienced in the early 2000s.'}"]}

In [33]:
for item in new_response['economist']:
    print(item['economist'])

['{\n  "improvement": "Y",\n  "response": "As of the latest updates up until 2023, China\'s GDP has shown signs of steady growth, though at a reduced pace compared to earlier decades. The slowdown can be attributed to a variety of economic challenges including demographic changes, increased debt ratios, and ongoing trade disagreements with key global players. The Chinese government has responded with a range of policies aimed at boosting domestic consumption, fostering technological innovation, and promoting sustainable economic activities to ensure long-term stability and continued expansion of the economy.",\n  "reason": "The improved response integrates insights from the other economists about factors influencing China\'s GDP, such as trade tensions and the COVID-19 pandemic, while emphasizing more on governmental measures to promote economic stability and sustainability.",\n  "further_improvement": "N"\n}']


In [27]:
for item in new_response:
    print(item)

economist


In [19]:
new_response

{'economist': [{'economist': ['{\n    "improvement": "Y",\n    "response": "As of the last data available up to 2023, China\'s GDP has displayed a pattern of gradual growth, although at a reduced pace compared to previous decades. The country confronts multiple economic challenges including demographic shifts, increased debt levels, and trade tensions with major economies. In response, the Chinese government has enacted policies focused on boosting domestic consumption, enhancing technological innovation, and fostering sustainable economic measures to secure stability and foster continuous development. Its growth trajectory has historically been strong; however, recent years have witnessed a moderation as the economy matures, transitioning towards a more service-oriented and consumption-driven market. Political and economic policies aimed at managing debt and restructuring the economy also play significant roles in shaping the GDP trends.",\n    "reason": "The revised response integrat

In [ ]:
new_response

In [ ]:
combined_response = {}
combined_response['data analyst'] = []
for item in new_response:
    
    # print(item['data analyst'])

    combined_response['data analyst'].extend(item['data analyst'])

print(combined_response)
    

In [ ]:
combined_response['data analyst'][0]